### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).  To get started, you can read more about this project and the dataset from the [publication here](http://crowdrec2013.noahlab.com.hk/papers/crowdrec2013_Dooms.pdf).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/movies.dat', 
                     delimiter='::', header=None, names=['movie_id', 'movie', 'genre'], 
                     dtype={'movie_id': object}, engine='python', encoding='utf-8')
reviews = pd.read_csv('https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/ratings.dat', 
                      delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], 
                      dtype={'movie_id': object, 'user_id': object, 'timestamp': object}, engine='python', encoding='utf-8')

#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [14]:
from IPython.display import display

# Initial overview of the data
display(movies.head(5))
display(reviews.head(5))

,movie_id,movie,genre
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short
1,0000010,La sortie des usines Lumière (1895),Documentary|Short
2,0000012,The Arrival of a Train (1896),Documentary|Short
3,25,The Oxford and Cambridge University Boat Race ...,NaN
4,0000091,Le manoir du diable (1896),Short|Horror


,user_id,movie_id,rating,timestamp
0,1,0111161,10,1373234211
1,1,0117060,7,1373415231
2,1,0120755,6,1373424360
3,1,0317919,6,1373495763
4,1,0454876,10,1373621125


In [16]:
display(movies.describe())
display(reviews.describe())

,movie_id,movie,genre
count,32875,32875,32603
unique,32875,32824,1240
top,3813916,Home (2016),Drama
freq,1,4,3749


,rating
count,760969.000000
mean,7.298560
std,1.854655
min,0.000000
25%,6.000000
50%,7.000000
75%,9.000000
max,10.000000


In [21]:
movies.isna().sum()

movie_id      0
movie         0
genre       272
dtype: int64

In [28]:
reviews.isna().any()

user_id      False
movie_id     False
rating       False
timestamp    False
dtype: bool

#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's

#### Reviews
* Create a date out of time stamp

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

In [73]:
import re

# Create new date column
movies['date'] = movies.loc[:, :].apply(lambda row: re.match('.*\((.*)\)', row.movie).groups()[0], axis=1)

In [ ]:
# and a few more below, which you can use as necessary

In [70]:
reviews_new, movies_new = t.show_clean_dataframes()

   Unnamed: 0  user_id  movie_id  rating   timestamp                 date  \
0           0        1     68646      10  1381620027  2013-10-12 23:20:27   
1           1        1    113277      10  1379466669  2013-09-18 01:11:09   
2           2        2    422720       8  1412178746  2014-10-01 15:52:26   
3           3        2    454876       8  1394818630  2014-03-14 17:37:10   
4           4        2    790636       7  1389963947  2014-01-17 13:05:47   

   month_1  month_2  month_3  month_4    ...      month_9  month_10  month_11  \
0        0        0        0        0    ...            0         1         0   
1        0        0        0        0    ...            0         0         0   
2        0        0        0        0    ...            0         1         0   
3        0        0        0        0    ...            0         0         0   
4        0        0        0        0    ...            0         0         0   

   month_12  year_2013  year_2014  year_2015  year